## Setup

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from datetime import timedelta
import random

In [2]:
# Cambiamos el directorio para importar las funciones de func_aux
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).absolute().parent))

In [3]:
from src.funcs_aux import *

In [4]:
datos_unidos = pd.read_csv("../data/procesados/datos_unidos.csv")

In [5]:
datos_unidos.columns

Index(['SKU', 'DATE', 'STORE_ID', 'PRICE', 'QUANTITY', 'TOTAL_SALES', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CLOSEDATE',
       'STORE_SUBGROUP_DATE_ID', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'INITIAL_TICKET_PRICE', 'BASE_PRICE',
       'COSTOS', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK'],
      dtype='object')

In [6]:
cols_categoricas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE',  'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', "DAY_OF_WEEK"]

target = "TOTAL_SALES"

In [7]:
for col in cols_categoricas:
    datos_unidos[col] = datos_unidos[col].astype("category")

In [8]:
datos_unidos.sort_values(by=["DATE", "STORE_ID", "SKU"], inplace=True)

## Feature aggregation

Primero, haremos el promedio de ventas por SKU X STORE_ID de los últimos 7, 30 y 90 días, para compensar el hecho de que no es posible (por limitaciones computacionales) completar todo el dataset con los días en que no hubo transacciones de un producto.

De esta manera, el modelo podrá dilucidar las épocas en donde no hay ventas de ciertos productos.

In [9]:
# datos_unidos = rolling_sales_completo(datos_unidos)

Aclaración: finalmente, en los modelos no utilizamos estos últimos features dado que elevaban el costo computacional y los modelos no presentaban una mejora significativa de performance con ellos

Ahora, haremos el promedio y desviación estándar de las ventas por subgrupo y por categoría, de manera que el modelo pueda entender mejor los cambios de ventas por épocas del año de grupos más grandes de productos.

In [9]:
rolling_sales(datos_unidos, group="SUBGROUP", windows=[30, 90], std=False)
rolling_sales(datos_unidos, group="SKU", windows=[30, 90], std=False)
rolling_sales(datos_unidos, group="STORE_ID", windows=[30, 90], std=False)

c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  datos_unidos.groupby([group, 'DATE'], as_index=False)['TOTAL_SALES']
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  subgroup_daily.groupby(group)['TOTAL_SALES']
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or ob

In [10]:
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SKU", windows=[30,90], std=False)
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SUBGROUP", windows=[30,90], std=False)

c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  datos_unidos.groupby('SKU')['PRICE']
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:74: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  datos_unidos.groupby([group, 'DATE'], as_index=False)['price_pct_change']
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:81: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

## Test de modelos

### LightGBM

In [11]:
import lightgbm as lgb

In [12]:
def walk_forward_lightgbm(df, features, target_col, date_col, categorical_cols,
                          train_days=365, step_days=30, forecast_days=7,
                          params=None):
    """
    df: DataFrame con features + target
    target_col: nombre de la columna objetivo (ej. 'TOTAL_SALES')
    date_col: columna con la fecha
    categorical_cols: lista de columnas categóricas (deben ser dtype 'category')
    train_days, step_days, forecast_days: enteros en días
    params: dict de parámetros LightGBM
    """

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)

    results = []
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    start_train_end = min_date + timedelta(days=train_days)

    count = 0

    while start_train_end + timedelta(days=forecast_days) <= max_date:
        count+=1

        print(f"Walk-forward: iteracion numero {count}")

        # Train y Test
        train_data = df[df[date_col] < start_train_end]
        test_data = df[(df[date_col] >= start_train_end) &
                       (df[date_col] < start_train_end + timedelta(days=forecast_days))]

        if len(test_data) == 0:
            break

        # Creamos un validation set para early stopping
        valid_days_inner = 7
        train_end_inner = train_data["DATE"].max() - timedelta(days=valid_days_inner)

        train_inner = train_data[train_data["DATE"] <= train_end_inner]
        valid_inner = train_data[train_data["DATE"] > train_end_inner]

        X_train_inner = train_inner[features]
        y_train_inner = train_inner[target_col]
        X_valid_inner = valid_inner[features]
        y_valid_inner = valid_inner[target_col]

        # Dataset LightGBM
        lgb_train = lgb.Dataset(X_train_inner, label=y_train_inner, categorical_feature=categorical_cols)
        lgb_valid = lgb.Dataset(X_valid_inner, label=y_valid_inner, categorical_feature=categorical_cols, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_valid],
            valid_names=["train_inner", "valid_inner"]
        )

        # Predicciones
        y_test_pred = model.predict(test_data[features], num_iteration=model.best_iteration)
        y_train_pred = model.predict(X_train_inner,num_iteration=model.best_iteration)
        
        # Métricas
        r2_test = r2_score(test_data[target], y_test_pred)
        r2_train = r2_score(y_train_inner, y_train_pred)


        results.append({
            "train_end_date": start_train_end,
            "r2_train": r2_train,
            "r2_test": r2_test
        })

        start_train_end += timedelta(days=step_days)

    return pd.DataFrame(results)

In [13]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

['SKU', 'STORE_ID', 'PRICE', 'REGION', 'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK', 'SUBGROUP_mean_30D', 'SUBGROUP_std_30D', 'SUBGROUP_mean_90D', 'SUBGROUP_std_90D', 'SUBGROUP_mean_180D', 'SUBGROUP_std_180D', 'SKU_mean_30D', 'SKU_std_30D', 'SKU_mean_90D', 'SKU_std_90D', 'SKU_mean_180D', 'SKU_std_180D', 'STORE_ID_mean_30D', 'STORE_ID_std_30D', 'STORE_ID_mean_90D', 'STORE_ID_std_90D', 'STORE_ID_mean_180D', 'STORE_ID_std_180D']


In [14]:
results_lgb = walk_forward_lightgbm(
    df=datos_unidos,
    features=features,
    target_col="TOTAL_SALES",
    date_col="DATE",
    categorical_cols=cols_categoricas,
    train_days=365,
    step_days=30,
    forecast_days=7,
    params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 2,
        "learning_rate": 0.01,
        "num_leaves": 400,
        "max_depth": 20,
        "min_data_in_leaf": 50,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 5,
        "early_stopping_round": 30,
        "num_boost_round":1000
    }
)

Walk-forward: iteracion numero 1
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.950365
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.114499
[LightGBM] [Debug] init for col-wise cost 0.067873 seconds, init for row-wise cost 0.428935 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.568096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6234
[LightGBM] [Info] Number of data points in the train set: 2242841, number of used features: 39
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score 299.483629
[LightGBM] [Debug] Re-bagging, using 2019054 data to train
[LightGBM] [Debug] Trained a tre

In [17]:
results_lgb.to_csv("resultados_test/resultados_lgb6.csv")

In [18]:
results_lgb.mean()

train_end_date    2022-12-27 00:00:00
r2_train                     0.746958
r2_test                      0.692159
dtype: object

## Deploy

### Training

In [11]:
import lightgbm as lgb

In [12]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

['SKU', 'STORE_ID', 'PRICE', 'REGION', 'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK', 'SUBGROUP_mean_30D', 'SUBGROUP_mean_90D', 'SKU_mean_30D', 'SKU_mean_90D', 'STORE_ID_mean_30D', 'STORE_ID_mean_90D', 'SKU_price_pct_mean_30D', 'SKU_price_pct_mean_90D', 'SUBGROUP_price_pct_mean_30D', 'SUBGROUP_price_pct_mean_90D']


In [ ]:
params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 1,
        "learning_rate": 0.01,
        "num_leaves": 300,
        "max_depth": 20,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 1,
		"num_boost_round" : 500,
    }

In [14]:
data_train = lgb.Dataset(datos_unidos[features], datos_unidos["TOTAL_SALES"], categorical_feature=cols_categoricas)

In [15]:
model_lgb = lgb.train(params, data_train)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.357953 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4199
[LightGBM] [Info] Number of data points in the train set: 6839616, number of used features: 32
[LightGBM] [Info] Start training from score 266.144489


### Template dataframe

Primero, creamos un dataframe base con todos los productos y tiendas para los 7 días, de manera que el modelo pueda predecir las ventas de cada combinación

In [16]:
template = crear_template(datos_unidos)

c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:511: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["SKU", "STORE_ID"], as_index=False)


In [17]:
# Como el modelo lo entrenaremos con 152 tiendas, pero dos de ellas cerraron y solo haremos la predicción de 150 tiendas, 
# necesitamos ajustar las categorias para que funcione correctamente lgb
type_stores = pd.api.types.CategoricalDtype(categories=datos_unidos["STORE_ID"].unique())
template["STORE_ID"] = template["STORE_ID"].astype(type_stores)

In [18]:
template = template.sort_values(by=["DATE", "STORE_ID", "SKU"])

In [19]:
template = rolling_price_template(datos_unidos, template, "SUBGROUP", windows=[30,90], std=False)
template = rolling_price_template(datos_unidos, template, "SKU", windows=[30,90], std=False)

c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:426: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('SKU')['PRICE']
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:443: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(group)["tem_price_pct_change"]
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:451: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and 

In [20]:
template = rolling_sales_template(df=datos_unidos, template=template, group= "SUBGROUP", windows=[30, 90], std=False)
template = rolling_sales_template(df=datos_unidos, template=template, group= "SKU", windows=[30, 90], std=False)
template = rolling_sales_template(df=datos_unidos, template=template, group= "STORE_ID", windows=[30, 90], std=False)

c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:348: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(group)["TOTAL_SALES"]
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:356: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(group)
c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\src\funcs_aux.py:348: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning

In [21]:
datos_unidos.columns

Index(['SKU', 'DATE', 'STORE_ID', 'PRICE', 'QUANTITY', 'TOTAL_SALES', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CLOSEDATE',
       'STORE_SUBGROUP_DATE_ID', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'INITIAL_TICKET_PRICE', 'BASE_PRICE',
       'COSTOS', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK', 'SUBGROUP_mean_30D',
       'SUBGROUP_mean_90D', 'SKU_mean_30D', 'SKU_mean_90D',
       'STORE_ID_mean_30D', 'STORE_ID_mean_90D', 'SKU_price_pct_mean_30D',
       'SKU_price_pct_mean_90D', 'SUBGROUP_price_pct_mean_30D',
       'SUBGROUP_price_pct_mean_90D'],
      dtype='object')

In [22]:
template.columns

Index(['DATE', 'SKU', 'STORE_ID', 'REGION', 'CITY', 'STATE', 'STORE_TYPE',
       'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID',
       'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'COSTOS', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK',
       'SUBGROUP_price_pct_mean_30D', 'SUBGROUP_price_pct_mean_90D',
       'SKU_price_pct_mean_30D', 'SKU_price_pct_mean_90D', 'SUBGROUP_mean_30D',
       'SUBGROUP_mean_90D', 'SKU_mean_30D', 'SKU_mean_90D',
       'STORE_ID_mean_30D', 'STORE_ID_mean_90D'],
      dtype='object')

### Optimizacion de precios

In [23]:
def crear_price_grid_descuento(datos_unidos: pd.DataFrame, n_prices: int = 50, descuento = 0.3) -> dict:
    """
    Devuelve dict { SKU: array de precios posibles }
    usando el último precio por SKU ±30%.
    """
    # Tomamos el último precio por SKU según la fecha
    ultimos_precios = datos_unidos.sort_values("DATE").groupby("SKU")["PRICE"].last().reset_index()

    price_grid = {}
    for _, r in ultimos_precios.iterrows():
        sku = r["SKU"]
        last_price = r["PRICE"]

        if pd.isna(last_price) or last_price <= 0:
            continue  # Evitamos precios nulos o negativos

        min_p = last_price * (1-descuento)  # -30%
        max_p = last_price * (1+descuento) # +30%

        price_grid[sku] = np.linspace(min_p, max_p, n_prices)

    return price_grid


In [24]:
price_grid = crear_price_grid_descuento(datos_unidos, n_prices=10, descuento = 0.2)

C:\Users\Juan Cruz\AppData\Local\Temp\ipykernel_16036\2767470758.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ultimos_precios = datos_unidos.sort_values("DATE").groupby("SKU")["PRICE"].last().reset_index()


In [ ]:
import optuna
import os

def optimizacion_precios_optuna(template, model, price_grid, features, target="GAIN", 
                                predict_func=None, save_dir=None, file_name="mejor_config", 
                                n_trials=1000, n_jobs=1, seed=42):
    """
    Optimiza la configuración de precios usando Optuna para maximizar GAIN o TOTAL_SALES,
    eligiendo un único precio por SKU.
    """
    np.random.seed(seed)

    # Lista de SKUs únicos
    skus_unicos = template["SKU"].unique()
    n_skus = len(skus_unicos)

    # Precalcular estructuras
    costos = template["COSTOS"].values
    precios_dict = {sku: np.array(price_grid[sku]) for sku in skus_unicos}

    # Mapear SKU -> posiciones en template (para asignar rápido)
    sku_posiciones = {sku: np.where(template["SKU"].values == sku)[0] for sku in skus_unicos}

    X_temp = template.copy()

    # Función objetivo para Optuna
    def objective(trial):
        precios_asignados = np.zeros(template.shape[0], dtype=float)

        # Elegir un precio por SKU y asignarlo a todas sus posiciones
        for sku in skus_unicos:
            opciones_precio = precios_dict[sku]
            idx = trial.suggest_int(f"sku_{sku}", 0, len(opciones_precio) - 1)
            precios_asignados[sku_posiciones[sku]] = opciones_precio[idx]

        # Predicción
        X_temp["PRICE"] = precios_asignados
        
        if predict_func is None:
            y_pred = model.predict(X_temp[features])
        else:
            y_pred = predict_func(model, X_temp[features])

        # Cálculos de métricas
        gain = y_pred.sum() - ((y_pred / precios_asignados) * costos).sum()
        total_sales = y_pred.sum()

        if target == "GAIN":
            return gain
        elif target == "TOTAL_SALES":
            return total_sales
        else:
            raise ValueError("El parámetro target debe ser 'GAIN' o 'TOTAL_SALES'.")

    # Crear estudio Optuna
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, show_progress_bar=True)

    # Obtener mejor configuración
    best_params = study.best_params
    precios_finales = np.zeros(template.shape[0], dtype=float)
    for sku in skus_unicos:
        precio = precios_dict[sku][best_params[f"sku_{sku}"]]
        precios_finales[sku_posiciones[sku]] = precio

    # Predecir con la mejor configuración
    X_best = template.copy()
    X_best["PRICE"] = precios_finales
    if predict_func is None:
        y_best = model.predict(X_best[features])
    else:
        y_best = predict_func(model, X_best[features])

    X_best["TOTAL_SALES"] = y_best

    mejor_gain = y_best.sum() - ((y_best / precios_finales) * costos).sum()
    mejor_sales = y_best.sum()

    # Guardar si es necesario
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        df_mejor = X_best.copy()
        df_mejor["TOTAL_SALES"] = y_best
        df_mejor.to_csv(f"{save_dir}/{file_name}.csv", index=False)

    return X_best, y_best, mejor_sales, mejor_gain, precios_finales, study


c:\Users\Juan Cruz\Desktop\GitHub\AlixPartners-Competencia\alix_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
df_best, y_best, mejor_sales, mejor_gain, precios_finales, study = optimizacion_precios_optuna(template=template, model=model_lgb, price_grid=price_grid,
                                                                                      features=features, target="GAIN", file_name="mejor_config_optuna",
                                                                                      n_trials=50, seed=42)

[I 2025-08-18 09:39:51,851] A new study created in memory with name: no-name-42067857-ecad-4c03-b9e0-b81bce706ae6
Best trial: 0. Best value: 8.08222e+07:   2%|▏         | 1/50 [00:03<02:59,  3.67s/it]

[I 2025-08-18 09:39:55,486] Trial 0 finished with value: 80822196.38363826 and parameters: {'sku_BEAHASH001': 3, 'sku_BEAHASH002': 9, 'sku_BEAHASH003': 7, 'sku_BEAHASH004': 5, 'sku_BEAHASH005': 1, 'sku_BEAHASH006': 1, 'sku_BEAHASH007': 0, 'sku_BEAHASH008': 8, 'sku_BEAHASH009': 6, 'sku_BEAHASH010': 7, 'sku_BEAHAST001': 0, 'sku_BEAHAST002': 9, 'sku_BEAHAST003': 8, 'sku_BEAHAST004': 2, 'sku_BEAHAST005': 1, 'sku_BEAHAST006': 1, 'sku_BEAHAST007': 3, 'sku_BEAHAST008': 5, 'sku_BEAHAST009': 4, 'sku_BEAHAST010': 2, 'sku_BEAHAST011': 6, 'sku_BEAHAST012': 1, 'sku_BEAHATR001': 2, 'sku_BEAHATR002': 3, 'sku_BEAHATR003': 4, 'sku_BEAHATR004': 7, 'sku_BEAHATR005': 1, 'sku_BEAHATR006': 5, 'sku_BEAHATR007': 5, 'sku_BEAHATR008': 0, 'sku_BEAMAEY001': 6, 'sku_BEAMAEY002': 1, 'sku_BEAMAEY003': 0, 'sku_BEAMAEY004': 9, 'sku_BEAMAEY005': 9, 'sku_BEAMAEY006': 8, 'sku_BEAMAEY007': 3, 'sku_BEAMAEY008': 0, 'sku_BEAMAEY009': 6, 'sku_BEAMAEY010': 4, 'sku_BEAMAEY011': 1, 'sku_BEAMAEY012': 4, 'sku_BEAMAFA001': 0, 'sku_

Best trial: 0. Best value: 8.08222e+07:   4%|▍         | 2/50 [00:07<02:59,  3.73s/it]

[I 2025-08-18 09:39:59,217] Trial 1 finished with value: 79961254.58986026 and parameters: {'sku_BEAHASH001': 2, 'sku_BEAHASH002': 1, 'sku_BEAHASH003': 0, 'sku_BEAHASH004': 4, 'sku_BEAHASH005': 5, 'sku_BEAHASH006': 0, 'sku_BEAHASH007': 7, 'sku_BEAHASH008': 4, 'sku_BEAHASH009': 5, 'sku_BEAHASH010': 4, 'sku_BEAHAST001': 4, 'sku_BEAHAST002': 5, 'sku_BEAHAST003': 1, 'sku_BEAHAST004': 1, 'sku_BEAHAST005': 8, 'sku_BEAHAST006': 9, 'sku_BEAHAST007': 3, 'sku_BEAHAST008': 2, 'sku_BEAHAST009': 6, 'sku_BEAHAST010': 4, 'sku_BEAHAST011': 0, 'sku_BEAHAST012': 1, 'sku_BEAHATR001': 7, 'sku_BEAHATR002': 6, 'sku_BEAHATR003': 0, 'sku_BEAHATR004': 2, 'sku_BEAHATR005': 2, 'sku_BEAHATR006': 6, 'sku_BEAHATR007': 0, 'sku_BEAHATR008': 1, 'sku_BEAMAEY001': 7, 'sku_BEAMAEY002': 1, 'sku_BEAMAEY003': 6, 'sku_BEAMAEY004': 2, 'sku_BEAMAEY005': 0, 'sku_BEAMAEY006': 2, 'sku_BEAMAEY007': 7, 'sku_BEAMAEY008': 8, 'sku_BEAMAEY009': 8, 'sku_BEAMAEY010': 3, 'sku_BEAMAEY011': 6, 'sku_BEAMAEY012': 2, 'sku_BEAMAFA001': 2, 'sku_

Best trial: 2. Best value: 8.13028e+07:   6%|▌         | 3/50 [00:12<03:17,  4.20s/it]

...

[I 2025-08-18 09:46:41,599] Trial 47 finished with value: 83658982.0613398 and parameters: {'sku_BEAHASH001': 2, 'sku_BEAHASH002': 8, 'sku_BEAHASH003': 8, 'sku_BEAHASH004': 8, 'sku_BEAHASH005': 2, 'sku_BEAHASH006': 0, 'sku_BEAHASH007': 8, 'sku_BEAHASH008': 3, 'sku_BEAHASH009': 3, 'sku_BEAHASH010': 8, 'sku_BEAHAST001': 5, 'sku_BEAHAST002': 7, 'sku_BEAHAST003': 6, 'sku_BEAHAST004': 8, 'sku_BEAHAST005': 5, 'sku_BEAHAST006': 4, 'sku_BEAHAST007': 8, 'sku_BEAHAST008': 6, 'sku_BEAHAST009': 9, 'sku_BEAHAST010': 0, 'sku_BEAHAST011': 6, 'sku_BEAHAST012': 2, 'sku_BEAHATR001': 3, 'sku_BEAHATR002': 0, 'sku_BEAHATR003': 6, 'sku_BEAHATR004': 6, 'sku_BEAHATR005': 5, 'sku_BEAHATR006': 3, 'sku_BEAHATR007': 3, 'sku_BEAHATR008': 7, 'sku_BEAMAEY001': 6, 'sku_BEAMAEY002': 2, 'sku_BEAMAEY003': 2, 'sku_BEAMAEY004': 7, 'sku_BEAMAEY005': 0, 'sku_BEAMAEY006': 4, 'sku_BEAMAEY007': 6, 'sku_BEAMAEY008': 2, 'sku_BEAMAEY009': 8, 'sku_BEAMAEY010': 9, 'sku_BEAMAEY011': 0, 'sku_BEAMAEY012': 2, 'sku_BEAMAFA001': 8, 'sku_

Best trial: 42. Best value: 8.45932e+07:  98%|█████████▊| 49/50 [07:02<00:10, 10.76s/it]

[I 2025-08-18 09:46:54,647] Trial 48 finished with value: 83932562.61615118 and parameters: {'sku_BEAHASH001': 4, 'sku_BEAHASH002': 5, 'sku_BEAHASH003': 9, 'sku_BEAHASH004': 9, 'sku_BEAHASH005': 4, 'sku_BEAHASH006': 1, 'sku_BEAHASH007': 9, 'sku_BEAHASH008': 8, 'sku_BEAHASH009': 5, 'sku_BEAHASH010': 7, 'sku_BEAHAST001': 8, 'sku_BEAHAST002': 8, 'sku_BEAHAST003': 0, 'sku_BEAHAST004': 7, 'sku_BEAHAST005': 3, 'sku_BEAHAST006': 1, 'sku_BEAHAST007': 6, 'sku_BEAHAST008': 3, 'sku_BEAHAST009': 8, 'sku_BEAHAST010': 2, 'sku_BEAHAST011': 4, 'sku_BEAHAST012': 5, 'sku_BEAHATR001': 5, 'sku_BEAHATR002': 3, 'sku_BEAHATR003': 4, 'sku_BEAHATR004': 4, 'sku_BEAHATR005': 7, 'sku_BEAHATR006': 5, 'sku_BEAHATR007': 4, 'sku_BEAHATR008': 9, 'sku_BEAMAEY001': 4, 'sku_BEAMAEY002': 4, 'sku_BEAMAEY003': 0, 'sku_BEAMAEY004': 4, 'sku_BEAMAEY005': 3, 'sku_BEAMAEY006': 3, 'sku_BEAMAEY007': 6, 'sku_BEAMAEY008': 0, 'sku_BEAMAEY009': 9, 'sku_BEAMAEY010': 2, 'sku_BEAMAEY011': 1, 'sku_BEAMAEY012': 0, 'sku_BEAMAFA001': 3, 'sku

Best trial: 42. Best value: 8.45932e+07: 100%|██████████| 50/50 [07:13<00:00,  8.68s/it]


[I 2025-08-18 09:47:05,675] Trial 49 finished with value: 82758146.42193604 and parameters: {'sku_BEAHASH001': 8, 'sku_BEAHASH002': 6, 'sku_BEAHASH003': 6, 'sku_BEAHASH004': 6, 'sku_BEAHASH005': 7, 'sku_BEAHASH006': 3, 'sku_BEAHASH007': 8, 'sku_BEAHASH008': 7, 'sku_BEAHASH009': 8, 'sku_BEAHASH010': 5, 'sku_BEAHAST001': 7, 'sku_BEAHAST002': 7, 'sku_BEAHAST003': 5, 'sku_BEAHAST004': 6, 'sku_BEAHAST005': 4, 'sku_BEAHAST006': 5, 'sku_BEAHAST007': 6, 'sku_BEAHAST008': 2, 'sku_BEAHAST009': 7, 'sku_BEAHAST010': 4, 'sku_BEAHAST011': 5, 'sku_BEAHAST012': 0, 'sku_BEAHATR001': 2, 'sku_BEAHATR002': 4, 'sku_BEAHATR003': 5, 'sku_BEAHATR004': 5, 'sku_BEAHATR005': 1, 'sku_BEAHATR006': 4, 'sku_BEAHATR007': 5, 'sku_BEAHATR008': 5, 'sku_BEAMAEY001': 5, 'sku_BEAMAEY002': 6, 'sku_BEAMAEY003': 6, 'sku_BEAMAEY004': 6, 'sku_BEAMAEY005': 7, 'sku_BEAMAEY006': 1, 'sku_BEAMAEY007': 5, 'sku_BEAMAEY008': 5, 'sku_BEAMAEY009': 9, 'sku_BEAMAEY010': 7, 'sku_BEAMAEY011': 4, 'sku_BEAMAEY012': 1, 'sku_BEAMAFA001': 5, 'sku

### Guardar resultado

In [27]:
import itertools

def crear_csv_kaggle(df_best, dummy_subgroup='DUMMY'):
    """
    Construye un dataframe con todas las combinaciones STORE_ID x SUBGROUP x DATE,
    incluyendo un subgrupo dummy con TOTAL_SALES=0, y suma TOTAL_SALES de df_best.
    
    Parámetros:
    -----------
    df_best : pd.DataFrame
        Debe contener columnas: DATE, STORE_ID, SUBGROUP, TOTAL_SALES
    dummy_subgroup : str
        Nombre del subgrupo que siempre tendrá TOTAL_SALES=0
    
    Retorna:
    --------
    df_result : pd.DataFrame
        Columnas: STORE_SUBGROUP_DATE_ID, TOTAL_SALES
    """

    # Asegurar formato de DATE
    df_best = df_best.copy()
    df_best['DATE'] = pd.to_datetime(df_best['DATE'])

    # Valores únicos de stores y subgrupos (agregamos dummy)
    stores = df_best['STORE_ID'].unique()
    subgroups = df_best['SUBGROUP'].unique().tolist() + [dummy_subgroup]
    dates = df_best['DATE'].unique()

    # Crear todas las combinaciones posibles
    all_combinations = pd.DataFrame(
        list(itertools.product(stores, subgroups, dates)),
        columns=['STORE_ID','SUBGROUP','DATE']
    )

    # Agrupar df_best por STORE_ID, SUBGROUP, DATE y sumar TOTAL_SALES
    df_agg = (
        df_best.groupby(['STORE_ID','SUBGROUP','DATE'], as_index=False)['TOTAL_SALES']
        .sum()
    )

    # Merge con todas las combinaciones
    df_result = all_combinations.merge(df_agg, on=['STORE_ID','SUBGROUP','DATE'], how='left')

    # Rellenar faltantes con 0 (incluye automáticamente dummy)
    df_result['TOTAL_SALES'] = df_result['TOTAL_SALES'].fillna(0)

    # Crear STORE_SUBGROUP_DATE_ID
    df_result['STORE_SUBGROUP_DATE_ID'] = (
        df_result['STORE_ID'].astype(str) + '_' +
        df_result['SUBGROUP'].astype(str) + '_' +
        df_result['DATE'].astype(str)
    )

    # Dejar solo las columnas finales
    df_result = df_result[['STORE_SUBGROUP_DATE_ID','TOTAL_SALES']]

    return df_result


In [28]:
df_result = crear_csv_kaggle(df_best, dummy_subgroup="Basketball")

C:\Users\Juan Cruz\AppData\Local\Temp\ipykernel_16036\789492514.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_best.groupby(['STORE_ID','SUBGROUP','DATE'], as_index=False)['TOTAL_SALES']


In [29]:
df_result.to_csv("resultados_optimizacion/mejor_df.csv", index=False)

## Test

In [100]:
features_subgroup = ["REGION", "CATEGORY", "GROUP"]

In [102]:
import pandas as pd

# 1. Agrupar las ventas
df_grouped = (
    datos_unidos
    .groupby(['SUBGROUP', 'STORE_ID', 'DATE'], as_index=False)["TOTAL_SALES"].sum()
)

# Asegurar formato datetime
df_grouped['DATE'] = pd.to_datetime(df_grouped['DATE'])

# 2. Crear rango completo de fechas
all_dates = pd.date_range(df_grouped['DATE'].min(), df_grouped['DATE'].max())

# 3. Obtener todas las combinaciones posibles SUBGROUP x STORE_ID
subgroup_store_pairs = (
    df_grouped[['SUBGROUP', 'STORE_ID']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 4. Cross join para todas las combinaciones y fechas
subgroup_store_pairs['key'] = 1
dates_df = pd.DataFrame({'DATE': all_dates, 'key': 1})
full_combinations = subgroup_store_pairs.merge(dates_df, on='key').drop(columns='key')

# Asegurar formato datetime
full_combinations['DATE'] = pd.to_datetime(full_combinations['DATE'])

# 5. Merge y completar con ceros
df_full = (
    full_combinations
    .merge(df_grouped, on=['SUBGROUP', 'STORE_ID', 'DATE'], how='left')
    .fillna({'TOTAL_SALES': 0})
)

df_full['TOTAL_SALES'] = df_full['TOTAL_SALES'].astype(float)

print(df_full.shape)
print(df_full.head())


C:\Users\Juan Cruz\AppData\Local\Temp\ipykernel_2796\2918020996.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['SUBGROUP', 'STORE_ID', 'DATE'], as_index=False)["TOTAL_SALES"].sum()


(12150120, 4)
      SUBGROUP STORE_ID       DATE  TOTAL_SALES
0  Accessories   000006 2021-01-01         0.00
1  Accessories   000006 2021-01-02         0.00
2  Accessories   000006 2021-01-03         0.00
3  Accessories   000006 2021-01-04       967.10
4  Accessories   000006 2021-01-05      1414.53


In [103]:
df_full["YEAR"] = df_full["DATE"].dt.year
df_full["MONTH"] = df_full["DATE"].dt.month
df_full["DAY"] = df_full["DATE"].dt.day
df_full["DAY_OF_WEEK"] = df_full["DATE"].dt.day_name()
df_full["WEEK"] = df_full["DATE"].dt.isocalendar().week

In [104]:
df_full.columns

Index(['SUBGROUP', 'STORE_ID', 'DATE', 'TOTAL_SALES', 'YEAR', 'MONTH', 'DAY',
       'DAY_OF_WEEK', 'WEEK'],
      dtype='object')

In [138]:
features_subgroup = ['SUBGROUP', 'STORE_ID', 'YEAR', 'MONTH', 'DAY',
       'DAY_OF_WEEK', 'WEEK']

In [111]:
for col in features_subgroup:
    df_full[col] = df_full[col].astype("category")

In [112]:
data_train = lgb.Dataset(df_full[features_subgroup], df_full["TOTAL_SALES"], categorical_feature=features_subgroup)

In [ ]:
params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 1,
        "learning_rate": 0.01,
        "num_leaves": 200,
        "max_depth": 20,
        "feature_fraction": 1,
        "bagging_fraction": 1,
        "bagging_freq": 0,
		"num_boost_round" : 100,
    }

In [113]:
model_lgb = lgb.train(params, data_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,152628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 336
[LightGBM] [Info] Number of data points in the train set: 12150120, number of used features: 7
[LightGBM] [Info] Start training from score 149,819599


In [164]:
template_subgroup = template[features_subgroup + ["DATE"]].drop_duplicates()

In [134]:
template_subgroup

,SUBGROUP,STORE_ID,YEAR,MONTH,DAY,DAY_OF_WEEK,WEEK,DATE
0,Shampoo,000006,2024,1,1,Monday,1,2024-01-01
10,Styling,000006,2024,1,1,Monday,1,2024-01-01
22,Treatment,000006,2024,1,1,Monday,1,2024-01-01
30,Eyes,000006,2024,1,1,Monday,1,2024-01-01
42,Face,000006,2024,1,1,Monday,1,2024-01-01
...,...,...,...,...,...,...,...,...
896657,Learning,STORE0149,2024,1,7,Sunday,1,2024-01-07
896664,STEM,STORE0149,2024,1,7,Sunday,1,2024-01-07
896672,Board Games,STORE0149,2024,1,7,Sunday,1,2024-01-07
896684,Card Games,STORE0149,2024,1,7,Sunday,1,2024-01-07


In [165]:
for col in features_subgroup:
    type_stores = pd.api.types.CategoricalDtype(categories=df_full[col].unique())
    template_subgroup[col] = template_subgroup[col].astype(type_stores)

In [166]:
pred_subgroup = model_lgb.predict(template_subgroup[features_subgroup])

In [167]:
template_subgroup["TOTAL_SALES"] = pred_subgroup

In [168]:
# Crear STORE_SUBGROUP_DATE_ID
template_subgroup['STORE_SUBGROUP_DATE_ID'] = (
        template_subgroup['STORE_ID'].astype(str) + '_' +
        template_subgroup['SUBGROUP'].astype(str) + '_' +
        template_subgroup['DATE'].astype(str)
    )

In [169]:
stores = template_subgroup['STORE_ID'].unique()
fechas = template_subgroup['DATE'].unique()

# Crear DataFrame con todas las combinaciones para el nuevo SUBGROUP
basketball_rows = pd.DataFrame([
    {
        'STORE_ID': store,
        'SUBGROUP': 'Basketball',
        'DATE': fecha,
        'TOTAL_SALES': 0,
        'STORE_SUBGROUP_DATE_ID': f"{store}_Basketball_{fecha.strftime('%Y-%m-%d')}"
    }
    for store in stores
    for fecha in fechas
])

# Concatenar con el template original
template_subgroup = pd.concat([template_subgroup, basketball_rows], ignore_index=True)

In [170]:
template_subgroup = template_subgroup[["STORE_SUBGROUP_DATE_ID", "TOTAL_SALES"]]

In [173]:
template_subgroup.to_csv("resultados_optimizacion/mejor_df.csv", index=False)

In [172]:
template_subgroup.sort_values(by="STORE_SUBGROUP_DATE_ID")

,STORE_SUBGROUP_DATE_ID,TOTAL_SALES
35,000006_Accessories_2024-01-01,357.864020
10985,000006_Accessories_2024-01-02,353.950789
21935,000006_Accessories_2024-01-03,355.862938
32885,000006_Accessories_2024-01-04,360.721921
43835,000006_Accessories_2024-01-05,357.952429
...,...,...
32814,STORE0149_Wearables_2024-01-03,418.595690
43764,STORE0149_Wearables_2024-01-04,440.947671
54714,STORE0149_Wearables_2024-01-05,434.822298
65664,STORE0149_Wearables_2024-01-06,437.524357
